In [1]:
import pandas as pd

# Given data
data = {
    "Prices": {
        "Monthly": {
            "Tax Upfront": 614.54,
            "Tax Included": 678.14,
            "Tax/Bank Included": 697.51,
            "1st Due": 725.41,
            "Sign & Drive": 749.56
        },
        "1st": {
            "Tax Upfront": 614.54,
            "Tax Included": 678.14,
            "Tax/Bank Included": 697.51,
            "1st Due": 725.41,
            "Sign & Drive": None
        },
        "Bank": {
            "Tax Upfront": 595.00,
            "Tax Included": 595.00,
            "Tax/Bank Included": None,
            "1st Due": None,
            "Sign & Drive": None
        },
        "DMV": {
            "Tax Upfront": 350.00,
            "Tax Included": 350.00,
            "Tax/Bank Included": 350.00,
            "1st Due": None,
            "Sign & Drive": None
        },
        "Doc": {
            "Tax Upfront": 249.00,
            "Tax Included": 249.00,
            "Tax/Bank Included": 249.00,
            "1st Due": None,
            "Sign & Drive": None
        },
        "Taxes": {
            "Tax Upfront": 2273.64,
            "Tax Included": 310.19,
            "Tax/Bank Included": 257.38,
            "1st Due": None,
            "Sign & Drive": None
        },
        "Shipping": {
            "Tax Upfront": 0.00,
            "Tax Included": 0.00,
            "Tax/Bank Included": 0.00,
            "1st Due": None,
            "Sign & Drive": None
        },
        "Cap. Red": {
            "Tax Upfront": 0.00,
            "Tax Included": 0.00,
            "Tax/Bank Included": 0.00,
            "1st Due": None,
            "Sign & Drive": None
        },
        "Exc. Fees": {
            "Tax Upfront": 0.00,
            "Tax Included": 0.00,
            "Tax/Bank Included": 0.00,
            "1st Due": None,
            "Sign & Drive": None
        },
        "Ref. Security Dep.": {
            "Tax Upfront": 0.00,
            "Tax Included": 0.00,
            "Tax/Bank Included": 0.00,
            "1st Due": None,
            "Sign & Drive": None
        },
        "Total": {
            "Tax Upfront": 4082.18,
            "Tax Included": 2182.33,
            "Tax/Bank Included": 1553.89,
            "1st Due": 725.41,
            "Sign & Drive": 0.00
        }
    }
}

# Convert the dictionary to a pandas DataFrame
df = pd.DataFrame(data["Prices"])

# Display the DataFrame
print(df)

                   Monthly     1st   Bank    DMV    Doc    Taxes  Shipping  \
Tax Upfront         614.54  614.54  595.0  350.0  249.0  2273.64       0.0   
Tax Included        678.14  678.14  595.0  350.0  249.0   310.19       0.0   
Tax/Bank Included   697.51  697.51    NaN  350.0  249.0   257.38       0.0   
1st Due             725.41  725.41    NaN    NaN    NaN      NaN       NaN   
Sign & Drive        749.56     NaN    NaN    NaN    NaN      NaN       NaN   

                   Cap. Red  Exc. Fees  Ref. Security Dep.    Total  
Tax Upfront             0.0        0.0                 0.0  4082.18  
Tax Included            0.0        0.0                 0.0  2182.33  
Tax/Bank Included       0.0        0.0                 0.0  1553.89  
1st Due                 NaN        NaN                 NaN   725.41  
Sign & Drive            NaN        NaN                 NaN     0.00  


In [3]:
df.transpose()

,Tax Upfront,Tax Included,Tax/Bank Included,1st Due,Sign & Drive
Monthly,614.54,678.14,697.51,725.41,749.56
1st,614.54,678.14,697.51,725.41,NaN
Bank,595.00,595.00,NaN,NaN,NaN
DMV,350.00,350.00,350.00,NaN,NaN
Doc,249.00,249.00,249.00,NaN,NaN
Taxes,2273.64,310.19,257.38,NaN,NaN
Shipping,0.00,0.00,0.00,NaN,NaN
Cap. Red,0.00,0.00,0.00,NaN,NaN
Exc. Fees,0.00,0.00,0.00,NaN,NaN
Ref. Security Dep.,0.00,0.00,0.00,NaN,NaN
